In [1]:
using Flux, Plots, Statistics, Random, ProgressMeter

In [2]:
Random.seed!(301341)

MersenneTwister(UInt32[0x0004991d], Random.DSFMT.DSFMT_state(Int32[-157111024, 1073331095, -867495699, 1073191104, 1639334311, 1073706424, -1758500981, 1073666098, 189956960, 1073687843  …  -1887146019, 1073447577, 494628161, 1073546910, 291638608, 1883496342, -415515204, 1815711982, 382, 0]), [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], UInt128[0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000  …  0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000

In [3]:
batchsize = 32


mnist_data = Flux.Data.MNIST.images();
mnist_labels = Flux.Data.MNIST.labels();

# clean mnist data
mnist_flat = [reshape(Float32.(img), :) for img in mnist_data];

# Check to make sure you know what batchseq does
all(get.(Flux.batchseq(mnist_flat[1:10]), 4, 0) .== mnist_flat[4])


data_loader = Flux.Data.DataLoader((mnist_flat, mnist_labels), batchsize=batchsize);



In [4]:
# create rnn
x_size = 1
num_labels = 10
num_hidden = 64
learning_rate = 0.001
num_epochs = 5000

rnn = Chain(LSTM(1, num_hidden), Dense(num_hidden, num_labels))
pred = (x)->softmax(rnn(x))

# create optimizer
opt = RMSProp(learning_rate)

RMSProp(0.001, 0.9, IdDict{Any,Any}())

In [11]:
ps = params(rnn)
for epoch ∈ 1:num_epochs
    sum_ℒ = 0.0f0
    prg = Progress("Epoch $(epoch): ", length(data_loader))
    for datum ∈ data_loader
        (X, Y) = datum
        x = reshape.((Flux.batchseq(X)), 1, :)
        y = Flux.batch([Flux.onehot(Y[i], 0:9) for i in 1:batchsize])
        Flux.reset!(rnn)
        gs = gradient(ps) do
            ℒ = Flux.Losses.logitcrossentropy(rnn.(x)[end], y)
            sum_ℒ += ℒ
            return ℒ
        end
#         func()
        Flux.update!(opt, ps, gs)
        next!(prg)
    end
    print("epoch: $(epoch), loss: $(sum_ℒ)")
end

MethodError: MethodError: no method matching Progress(::String, ::Int64)
Closest candidates are:
  Progress(!Matched::Integer, ::Real) at /Users/Matt/.julia/packages/ProgressMeter/poEzd/src/ProgressMeter.jl:95
  Progress(!Matched::Integer, ::Real, !Matched::AbstractString) at /Users/Matt/.julia/packages/ProgressMeter/poEzd/src/ProgressMeter.jl:95
  Progress(!Matched::Integer, ::Real, !Matched::AbstractString, !Matched::Any) at /Users/Matt/.julia/packages/ProgressMeter/poEzd/src/ProgressMeter.jl:95
  ...

In [8]:
length(data_loader)

1875